In [ ]:
import torch
import torch.nn as nn   
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import time
import matplotlib
import matplotlib.pyplot as plt
import random
import pandas as pd
import torch.nn.functional as F
device=torch.device('cpu')
cpac_root='/media/dm/0001A094000BF891/Yazid/ABIDEI_CPAC/'
smri_root='/media/dm/0001A094000BF891/Yazid/ABIDEI_sMRI/'
nan_subid=np.load('nan_subid.npy').tolist()
aal=np.load('Atlas/AAL.npy').tolist()
Lobe_aal=np.load('SA-result/Lobe_aal.npy',allow_pickle=True).tolist()
Lobe={}
Lobe['Central']=[1,2,57,58,17,18]
Lobe['Frontal']=[3,4,5,6,7,8,9,10,11,12,13,14,15,16,19,20,21,22,23,24,25,26,27,28,69,70]
Lobe['Temporal']=[79,80,81,82,85,86,89,90]
Lobe['Parietal']=[59,60,61,62,63,64,65,66,67,68]
Lobe['Occipital']=[43,44,45,46,47,48,49,50,51,52,53,54,55,56]
Lobe['Limbic']=[31,32,33,34,35,36,37,38,39,40,83,84,87,88]
Lobe['Insula']=[29,30]
Lobe['Subcortical']=[41,42,71,72,73,74,75,76,77,78]
Lobe['Cerebelum']=[91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108]
Lobe['Vermis']=[109,110,111,112,113,114,115,116]

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention,self).__init__()
        self.conv1=nn.Conv1d(in_channels=3,out_channels=3,kernel_size=1,padding=0)
        self.conv2=nn.Conv1d(in_channels=116,out_channels=116,kernel_size=1,padding=0)
        self.softmax=nn.Softmax(dim=-1)
    def forward(self,Z,X):
        batchsize,x_dim,x_c= X.size()
        batchsize,z_dim,z_c= Z.size()
        K=self.conv1(X.permute(0,2,1))# BS,x_c,x_dim
        Q=K.permute(0,2,1)# BS,x_dim,x_c
        V=self.conv2(Z.permute(0,2,1))# Bs,z_c,z_dim
        attention=self.softmax(torch.matmul(Q,K))#BS,x_dim,x_dim
        out=torch.bmm(attention,V).permute(0,2,1)#BS,z_dim,z_c
        return out
class NEResGCN(nn.Module):
    def __init__(self,layer):
        super(NEResGCN,self).__init__()
        self.layer =layer
        self.relu  =nn.ReLU()
        self.atten =nn.ModuleList([Attention() for i in range(layer)])
        self.norm_n=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.norm_e=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.node_w=nn.ParameterList([nn.Parameter(torch.randn((3,3),dtype=torch.float32)) for i in range(layer)])
        self.edge_w=nn.ParameterList([nn.Parameter(torch.randn((116,116),dtype=torch.float32)) for i in range(layer)])
        self.line_n=nn.ModuleList([nn.Sequential(nn.Linear(116*3,128),nn.ReLU(),nn.BatchNorm1d(128)) for i in range(layer+1)])
        self.line_e=nn.ModuleList([nn.Sequential(nn.Linear(116*116,128*3),nn.ReLU(),nn.BatchNorm1d(128*3)) for i in range(layer+1)])
        self.clase =nn.Sequential(nn.Linear(128*4*(self.layer+1),1024),nn.Dropout(0.2),nn.ReLU(),
                                   nn.Linear(1024,2))
        self.ones=nn.Parameter(torch.ones((116),dtype=torch.float32),requires_grad=False)
        self._initialize_weights()
    # params initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d,nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def normalized(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        A=A+torch.diag(self.ones)
        d=A.sum(1)
        D=torch.diag(torch.pow(d,-1))
        A=D.mm(A).reshape(1,116,116)
        for i in range(1,n):
            A1=Z[i,:,:]+torch.diag(self.ones)
            d=A1.sum(1)
            D=torch.diag(torch.pow(d,-1))
            A1=D.mm(A1).reshape(1,116,116)
            A=torch.cat((A,A1),0)
        return A
        
    def update_A(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        Value,_=torch.topk(torch.abs(A.view(-1)),int(116*116*0.2))
        A=(torch.abs(A)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
        A=A.reshape(1,116,116)
        for i in range(1,n):
            A2=Z[i,:,:]
            Value,_=torch.topk(torch.abs(A2.view(-1)),int(116*116*0.2))
            A2=(torch.abs(A2)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
            A2=A2.reshape(1,116,116)
            A=torch.cat((A,A2),0)
        return A
        
    def forward(self,X,Z):
        n=X.size()[0]
        XX=self.line_n[0](X.view(n,-1))
        ZZ=self.line_e[0](Z.view(n,-1))
        for i in range(self.layer):
            A=self.atten[i](Z,X)
            Z1=torch.matmul(A,Z)
            Z2=torch.matmul(Z1,self.edge_w[i])
            Z=self.relu(self.norm_e[i](Z2))+Z
            #Z.register_hook(grad_Z_hook)
            #feat_Z_hook(Z)
            ZZ=torch.cat((ZZ,self.line_e[i+1](Z.view(n,-1))),dim=1)
            X1=torch.matmul(A,X)
            X1=torch.matmul(X1,self.node_w[i])
            X=self.relu(self.norm_n[i](X1))+X
            #X.register_hook(grad_X_hook)
            #feat_X_hook(X)
            XX=torch.cat((XX,self.line_n[i+1](X.view(n,-1))),dim=1)
        XZ=torch.cat((XX,ZZ),1)
        y=self.clase(XZ)
        #print(self.clase[0].weight)
        return y
# def grad_X_hook(grad):
#     X_grad.append(grad)
# def feat_X_hook(X):
#     X_feat.append(X.detach())
# def grad_Z_hook(grad):
#     Z_grad.append(grad)
# def feat_Z_hook(Z):
#     Z_feat.append(Z.detach())

In [ ]:
def normalized(X):
    return (X-X.mean())/X.std()
def gradient(device,model,dataloader):
    model.eval()
    result_asd_x=np.zeros((116,3))
    result_asd_z=np.zeros((116,116))
    result_tdc_x=np.zeros((116,3))
    result_tdc_z=np.zeros((116,116))
    for (X,Z,A,label,sub_id) in dataloader:
        model.zero_grad()
        X=torch.autograd.Variable(X,requires_grad=True)
        x=X.to(device)
        Z=torch.autograd.Variable(Z,requires_grad=True)
        z=Z.to(device)
        A=torch.autograd.Variable(A,requires_grad=True)
        a=A.to(device)
        y=model(x,z)
        if (label==torch.FloatTensor([0])).item():
            torch.autograd.backward(y,torch.FloatTensor([[1.,0.]]).to(device))
        else:
            torch.autograd.backward(y,torch.FloatTensor([[0.,1.]]).to(device))
        grad_X=X.grad.numpy()[0]
        grad_Z=Z.grad.numpy()[0]
        grad_X=normalized(grad_X)
        grad_Z=normalized(grad_Z)
        if label==torch.FloatTensor([0]).item():
            result_tdc_x+=grad_X
            result_tdc_z+=grad_Z
        else:
            result_asd_x+=grad_X
            result_asd_z+=grad_Z
    return result_asd_x,result_asd_z,result_tdc_x,result_tdc_z
def grad_cam(grad,feat,top_rate):
    N=len(grad)//5
    n=grad[0].shape[2]
    result=torch.zeros((116,n))
    for i in range(N):
        weight=torch.zeros(5)
        for j in range(5):
            weight[j]=(grad[i*5+j]*(grad[i*5+j]>0)).sum()
        weight=F.softmax(weight, dim=0)
        feature=torch.zeros((116,n))
        for j in range(5):
            feature+=weight[j]*(grad[i*5+j][0]>0)*grad[i*5+j][0]
        value_x,_=torch.topk(torch.abs(feature.view(-1)),int(116*n*top_rate))
        result+=(torch.abs(feature)>=value_x[-1])
    return result    

In [ ]:
def data_arange(sites,fmri_root,smri_root,nan_subid):
    asd,tdc=[],[]
    for site in sites:
        mri_asd=os.listdir(smri_root+site+'/group1')
        mri_tdc=os.listdir(smri_root+site+'/group2')
        fmri_asd=os.listdir(fmri_root+site+'/group1_FC')
        fmri_tdc=os.listdir(fmri_root+site+'/group2_FC')
        site_asd=[i for i in mri_asd if i in fmri_asd ]
        site_tdc=[i for i in mri_tdc if i in fmri_tdc ]
        site_asd=[i for i in site_asd if int(i[:5]) not in nan_subid]
        site_tdc=[i for i in site_tdc if int(i[:5]) not in nan_subid]
        asd.append(site_asd)
        tdc.append(site_tdc) 
    return asd,tdc
class dataset(Dataset):
    def __init__(self,fmri_root,smri_root,site,ASD,TDC,topk=True,rate=0.2):
        super(dataset,self).__init__()
        self.fmri=fmri_root
        self.smri=smri_root
        self.ASD=[j for i in ASD for j in i]
        self.TDC=[j for i in TDC for j in i]
        self.data=self.ASD+self.TDC
        random.shuffle(self.data)
        self.data_site={}
        for i in range(len(site)):
            data=ASD[i]+TDC[i]
            for j in data:
                if j not in self.data_site:
                    self.data_site[j]=site[i]                
        self.rate=rate
        self.topk=topk
    def normalize(self,A):
        d=A.sum(1)
        D=torch.diag(torch.pow(d,-1))
        return D.mm(A)
    def __getitem__(self,index):
        data=self.data[index]
        sub_id=int(data[0:5])
        if data in self.ASD:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group1_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group1_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group1/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group1_FC/'+data,allow_pickle=True)
        elif data in self.TDC:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group2_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group2_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group2/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group2_FC/'+data,allow_pickle=True)
        else:
            print('wrong input')
        data_slow5=(data_slow5-np.min(data_slow5))/(np.max(data_slow5)-np.min(data_slow5))
        data_slow4=(data_slow4-np.min(data_slow4))/(np.max(data_slow4)-np.min(data_slow4))
        if np.any(np.isnan(data_slow5)) or np.any(np.isnan(data_slow4)) or np.any(np.isnan(data_FCz)):
            print('data wronmg')
        #data_FCz=(data_FCz-np.min(data_FCz))/(np.max(data_FCz)-np.min(data_FCz))
        if self.data[index] in self.ASD:
            label=torch.tensor(1)
        else:
            label=torch.tensor(0)
        X=np.zeros((116,3),dtype=np.float32)
        X[:,0]=data_slow5
        X[:,1]=data_slow4
        X[:,2]=data_voxel
        data_FCz=data_FCz.astype(np.float32)
        Z=torch.from_numpy(data_FCz)
        X=torch.from_numpy(X)
        if self.topk:
            Value,_=torch.topk(torch.abs(Z.view(-1)),int(116*116*self.rate))
            A=(torch.abs(Z)>=Value[-1])+torch.tensor(0.0,dtype=torch.float32)
        A=self.normalize(A)
        return X,Z,A,label,sub_id
    def __len__(self):
        return len(self.data)

In [ ]:
## gradient-basedfor
featuresMap=list()
all_site=os.listdir(cpac_root)
Result_ASD_X,Result_TDC_X=np.zeros((116,3)),np.zeros((116,3))
Result_TDC_Z,Result_ASD_Z=np.zeros((116,116)),np.zeros((116,116))
vote = 13
for i in range(vote):
    for index in range(10):
        PATH='/media/dm/0001A094000BF891/Yazid/SAVEDModels/ensamble/models_{}_{}'.format(i,index)
        model=NEResGCN(5)
        model.load_state_dict(torch.load(PATH))
        train_asd,train_tdc=data_arange(all_site,fmri_root=cpac_root,smri_root=smri_root,nan_subid=nan_subid)
        trainset=dataset(site=all_site,fmri_root=cpac_root,smri_root=smri_root,ASD=train_asd,TDC=train_tdc)
        trainloader=DataLoader(trainset,batch_size=1,shuffle=True,drop_last=True)
        result_asd_x,result_asd_z,result_tdc_x,result_tdc_z=gradient(device,model,trainloader)
        Result_ASD_X+=result_asd_x
        Result_ASD_Z+=result_asd_z
        Result_TDC_X+=result_tdc_x
        Result_TDC_Z+=result_tdc_z
np.save('SA-result/ensamble/adTrained_asd_x_{}.npy'.format(vote),Result_ASD_X)
np.save('SA-result/ensamble/adTrained_asd_z_{}.npy'.format(vote),Result_ASD_Z)
np.save('SA-result/ensamble/adTrained_tdc_x_{}.npy'.format(vote),Result_TDC_X)
np.save('SA-result/ensamble/adTrained_tdc_z_{}.npy'.format(vote),Result_TDC_Z)